# Exercise 1: Connect to openLCA via olca-ipc

In [ ]:
import olca
import pandas as pd
pd.set_option('display.max_colwidth', None)

client = olca.Client(8080)
client 

# Exercise 2: Browse the Agribalyse database

In [ ]:
descriptor = client.get_descriptors(olca.Flow)#Process)#ImpactMethod)
elem_list = []
id_list = []

for elem in descriptor:
    elem_list.append(elem.name)
    id_list.append(elem.id)
df_all_elem = pd.DataFrame(list(zip(elem_list,
                                   id_list)),
                               columns=['name', 'id'])
df_all_elem 

In [ ]:
searcher = [elem for elem in df_all_elem['name'] if 'Tomato,' in elem
              and not 'conventional' in elem
           and 'dried' in elem]
searcher

In [ ]:
searchstring1 = 'wheat'
searchstring2 = 'flour'

In [ ]:
df_all_elem[(df_all_elem['name'].str.contains(searchstring1, regex=False, case=False)) & 
            (df_all_elem['name'].str.contains(searchstring2, regex=False, case=False))]

# Exercise 3: Creating new database elements

In [ ]:
from datetime import datetime

## Creating a new categories / folders
### Flow category

In [ ]:
new_cat = olca.Category()
new_cat.olca_type = 'Category'
new_cat.model_type = olca.ModelType['FLOW']
new_cat.name = '1: Tutorial flows'
client.insert(new_cat) 

In [ ]:
descriptor = client.get_descriptors(olca.Category)
instance_list = []
uuid_list = []

for element in descriptor:
    instance_list.append(element.name)
    uuid_list.append(element.id)
instance_df = pd.DataFrame(list(zip(instance_list,
                                   uuid_list)),
                               columns=['NAME', 'UUID'])
instance_df#.head(7)  

cache = instance_df.loc[instance_df['NAME'] == '1: Tutorial flows']['UUID'].values[0]

flow_cat = client.get(olca.Category, cache)
flow_cat

### Process category

In [ ]:
new_cat = olca.Category()
new_cat.olca_type = 'Category'
new_cat.model_type = olca.ModelType['PROCESS']
new_cat.name = '1: Tutorial processes - pizza production'
client.insert(new_cat) 

In [ ]:
descriptor = client.get_descriptors(olca.Category)
instance_list = []
uuid_list = []

for element in descriptor:
    instance_list.append(element.name)
    uuid_list.append(element.id)
instance_df = pd.DataFrame(list(zip(instance_list,
                                   uuid_list)),
                               columns=['NAME', 'UUID'])
instance_df#.head(7)  

cache = instance_df.loc[instance_df['NAME'] == '1: Tutorial processes - pizza production']['UUID'].values[0]

proc_cat = client.get(olca.Category, cache)
proc_cat

## Creating a new flow

In [ ]:
dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))

noitems = client.find(olca.FlowProperty, 'Number of items')

new_flow = olca.product_flow_of('pizza (tutorial)', noitems)
new_flow.description = 'Added via olca-ipc on %s.' % (dt_object)
new_flow.category = flow_cat
client.insert(new_flow) 

In [ ]:
new_flow

In [ ]:
mass = client.find(olca.FlowProperty, 'Mass')

new_flow2 = olca.product_flow_of('pizza (tutorial, mass)', mass)
new_flow2.description = 'Added via olca-ipc on %s.' % (dt_object)
new_flow2.category = flow_cat
client.insert(new_flow2)

## Creating a new process

In [ ]:
dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))

new_process = olca.process_of('pizza production (tutorial)')
new_process.description = 'Added via olca-ipc on %s.' % (dt_object)
new_process.category = proc_cat
client.insert(new_process)

## Creating new exchanges
### Creating the quantitative reference (output)

In [ ]:
list_exchanges = []
cache = client.get(olca.Flow, new_flow.id)
new_exchange = olca.output_of(new_process, cache, 10)
new_exchange.quantitative_reference = True
list_exchanges.append(new_exchange)

new_process.exchanges = list_exchanges
client.update(new_process) 

In [ ]:
list_exchanges 

### Reading in input-exchanges from prepared Excel-file

In [ ]:
df_pizza = pd.read_excel('pizza-process-characterization-tutorial.xlsx')
df_pizza 

In [ ]:
for index, row in df_pizza.iterrows():
    cache = client.get(olca.Flow, row['flow'])
    new_exchange = olca.input_of(new_process, cache, row['amount'])
    new_exchange.default_provider = client.get(olca.Process, row['provider'])

In [ ]:
list_exchanges 

In [ ]:
new_process.exchanges = list_exchanges
client.update(new_process) 

## Creating a new parameter

In [ ]:
cheese_param = olca.parameter_of('cheese_kg', 2.5, olca.ParameterScope.PROCESS_SCOPE)

new_process.parameters = [cheese_param]
client.update(new_process) 

In [ ]:
for ex in new_process.exchanges:
    if ex.flow.name == 'Cheese production; from raw milk, hard cheese ; French production mix, at plant; 1 kg of hard cheese (PGi)':
        ex.amount_formula = 'cheese_kg'
client.update(new_process) 

## Create a new product system

In [ ]:
product_system = client.create_product_system(new_process.id,
                                              default_providers='prefer',
                                              preferred_type='UNIT_PROCESS') 

# Exercise 4: Calculation
## Simple calculation

In [ ]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.UPSTREAM_ANALYSIS
setup.amount = 1
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013 GWP 100a')
setup.product_system = olca.ref(olca.ProductSystem,product_system.id)
#setup.allocation_method = olca.AllocationType.PHYSICAL_ALLOCATION

calc_result = client.calculate(setup)
gwp = calc_result.impact_results[0].value
print('The resulting GWP impact of making one pizza is %.2f kg CO2 eq.' % (gwp)) 
client.dispose(calc_result)  

## With parameter redefinition

In [ ]:
redef_list = [1,1.5,2,2.5,3] 

In [ ]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.UPSTREAM_ANALYSIS
setup.amount = 1
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013 GWP 100a')
setup.product_system = olca.ref(olca.ProductSystem,product_system.id)


result_category_list = []
result_value_list = []
result_unit_list = []

for x in range(len(redef_list)):
    redef = olca.ParameterRedef()
    redef.name = 'cheese_kg'
    redef.context = olca.ref(olca.Process, new_process.id)
    redef.value = redef_list[x]
    setup.parameter_redefs = [redef]
    
    calc_result = client.calculate(setup)
    gwp = calc_result.impact_results[0].value
    print('The resulting GWP impact of making a pizza with a cheese topping of %.2f kg,\t equals %.2f kg CO2 eq.' % (redef.value / 10, gwp))
    
    result_category_list.append(calc_result.impact_results[0].impact_category.name)
    result_value_list.append(calc_result.impact_results[0].value)
    result_unit_list.append(calc_result.impact_results[0].impact_category.ref_unit)

    client.dispose(calc_result) 

In [ ]:
redef_list_per_pizza = list(map(lambda x: x/10, redef_list))
redef_list_per_pizza 

In [ ]:
df_results = pd.DataFrame(list(zip(redef_list_per_pizza,
                                   result_value_list)),
                               columns=['parameter value: cheese amount per pizza', 'GWP result'])

df_results

In [ ]:
df_results.plot(x='parameter value: cheese amount per pizza', y='GWP result') 